In [1]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [3]:
%run -i ../../zkstats/core.py

In [4]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
comb_data_path = os.path.join('prover/comb_data.json')

=======================  ZK-STATS FLOW =======================

In [5]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

f_raw_input = open(data_path, "r")
data = json.loads(f_raw_input.read())["input_data"][0]
data_tensor = torch.reshape(torch.tensor(data),(1, len(data), 1))


dummy_data = np.round(np.random.uniform(1,10,len(data)),1)
json.dump({"input_data":[dummy_data.tolist()]}, open(dummy_data_path, 'w'))

dummy_data_tensor = torch.reshape(torch.tensor(dummy_data), (1, len(dummy_data),1 ))
dummy_theory_output = torch.exp(torch.mean(torch.log(dummy_data_tensor)))



In [6]:
# Verifier/ data consumer side:
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        self.w = nn.Parameter(data = dummy_theory_output, requires_grad = False)

    def forward(self,X):
        return (torch.abs((torch.log(self.w)*X.size()[1])-torch.sum(torch.log(X)))<=torch.log(torch.tensor(1.01)), self.w)

verifier_define_calculation(verifier_model, verifier_model_path, [dummy_data_path])

/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_45071/1643889957.py:8: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  return (torch.abs((torch.log(self.w)*X.size()[1])-torch.sum(torch.log(X)))<=torch.log(torch.tensor(1.01)), self.w)


In [7]:
# prover calculates settings, send to verifier
theory_output = torch.exp(torch.mean(torch.log(data_tensor)))
print("Theory output: ", theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = theory_output, requires_grad = False)

    def forward(self,X):
        return (torch.abs((torch.log(self.w)*X.size()[1])-torch.sum(torch.log(X)))<=torch.log(torch.tensor(1.01)), self.w)


prover_gen_settings([data_path], comb_data_path, prover_model,prover_model_path, [8], "resources", settings_path)

Theory output:  tensor(47.6981)
==== Generate & Calibrate Setting ====
scale:  [8]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":8,"param_scale":8,"scale_rebase_multiplier":10,"lookup_range":[-6,45056],"logrows":16,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":910,"total_const_size":4,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,8],"model_input_scales":[8],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":["Abs",{"Ln":{"scale":256.0}},{"GreaterThan":{"a":0.0}}],"check_mode":"UNSAFE","version":"7.0.0","num_blinding_factors":null}


/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_45071/21767248.py:11: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  return (torch.abs((torch.log(self.w)*X.size()[1])-torch.sum(torch.log(X)))<=torch.log(torch.tensor(1.01)), self.w)


In [8]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, comb_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

spawning module 0
spawning module 2


==== setting up ezkl ====


spawning module 0
spawning module 2


Time setup: 8.106843948364258 seconds
Theory output:  tensor(47.6981)
!@# compiled_model exists? True
!@# compiled_model exists? True
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 47.69921875
==== Generating Proof ====


spawning module 0
spawning module 2


proof:  {'instances': [[[12773529019820117503, 7223165032241168505, 6388811546753741567, 1325363978837834469], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [14148586052410868650, 7343294927740416096, 1182455714804702497, 77845129035718797]]], 'proof': '17f45df9955098f0ec8705b297fbfde9086ef44e1e4b5aa55c809ca505e3f1b204a94af68ee7b1394cdc3ecf18672bcf4b986acc132eff61dee2877d4e9b1e5d1651ddc2ff60ace015a798ee0f787be865b2b0f66f93f63b7b159d35a01b2b9c12946d54c6168658691f3acffbddea31076d66c24771205bc4aae871eb0478761905c51b9cdf89137aea5b0ee43d5b372620f57276bfa69095dd4b8145c9d09a2738ca64c51a1276dc372293b1aaba44de2c2cc129ff0dd6f5a2638779e2fdd81e9ba43e8b0725a376bc2768661e9dbe02334bb442c511ca846105b7aa420dcd1524899e25b99ce15b97040cf8e59a5655be2aaf62cd25e318bd2edea39f80132e03cccda8afbce284d97027b576ef9f9769b3db315bd35476d0f6190ccb0d9c189eed6abce26bc5fc209424ab8e52dac5be68b04d9e4b947154855467cc92780dd1268bb794caf9de44894ffa77d5ae3a2e43913a584b006aab538a56135a811

In [9]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path)

num_inputs:  1
prf instances:  [[[12773529019820117503, 7223165032241168505, 6388811546753741567, 1325363978837834469], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [14148586052410868650, 7343294927740416096, 1182455714804702497, 77845129035718797]]]
proof boolean:  1.0
proof result 1 : 47.69921875
verified
